In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

path = "../../resources/filters/"
xlims = (3000, 25000)

filter_sets = [
    {
        'title': 'HyperSurprimeCam (HSC) Filters',
        'bands': ["g", "r", "r2", "i", "i2", "z", "y"],
        'path_template': path + "filters_hsc_v2018/hsc_{}_v2018.dat",
    },
    {
        'title': 'DECam Filters (with Atmosphere)',
        'bands': ["u", "g", "r", "i", "z", "Y"],
        'path_template': path + "DECam_filters_with_atm.txt",
        'special_load': True,
    },
    {
        'title': 'DECam Filters (without Atmosphere)',
        'bands': ["u", "g", "r", "i", "z", "Y"],
        'path_template': path + "DECam_filters_with_atm.txt",
        'special_load': True,
        'normalize': True,
    },
    {
        'title': 'Bessel Filters',
        'bands': ["U", "B", "V", "R", "I", "J", "K"],
        'path_template': [
            path + "Generic_Bessell.{}.dat",
            path + "Generic_Bessell_JHKLM.{}.dat"
        ],
    },
    {
        'title': 'Johnson Filters',
        'bands': ["U", "B", "V", "R", "I", "J"],
        'path_template': path + "Generic_Johnson.{}.dat",
    },
    {
        'title': 'SDSS Filters',
        'bands': ["u", "g", "r", "i", "z"],
        'path_template': path + "filters_sdss/SLOAN_SDSS.{}.dat",
    },
    {
        'title': 'VISTA Filters (with Atmosphere)',
        'bands': ["Z", "Y", "J", "H", "Ks"],
        'path_template': path + "filters_vista/Paranal_VISTA.{}.dat",
    },
    {
        'title': 'VISTA Filters (without Atmosphere)',
        'bands': ["Z", "Y", "J", "H", "Ks"],
        'path_template': path + "filters_vista/Paranal_VISTA.{}_filter.dat",
    },
    {
        'title': 'SuprimeCam Filters',
        'bands': ["IA427", "IA464", "IA484", "IA505", "IA527", "IA574", "IA624", "IA679", "IA709", "IA738", "IA767", "IA827", "NB711", "NB816"],
        'path_template': path + "filters_cosmos2015/{}.SuprimeCam.pb",
    }
]

# Create subplots in a single column
plt.figure(figsize=(7, 25))
plt.subplots_adjust(hspace=0.4, wspace=0.3)

# Plot each filter set
for i, filter_set in enumerate(filter_sets, 1):
    plt.subplot(len(filter_sets), 1, i)
    plt.title(filter_set['title'])
    
    # Handle special loading for DECam
    if filter_set.get('special_load', False):
        filter_data = np.loadtxt(filter_set['path_template']).T
        lam_u, u = np.loadtxt(path+"DECam_2014_u.res").T
        
        for j, b in enumerate(filter_set['bands']):
            if j == 0:
                if not filter_set.get('normalize', False):
                    plt.plot(lam_u, u, label=f"DECam_{b}")
                continue
            
            if filter_set.get('normalize', False):
                plt.plot(filter_data[0], filter_data[j]/filter_data[-1], label=f"DECamNoAtm_{b}")
            else:
                plt.plot(filter_data[0], filter_data[j], label=f"DECam_{b}")
    
    else:
        for b in filter_set['bands']:
            try:
                # Try different path templates for Bessel filters
                if isinstance(filter_set['path_template'], list):
                    for template in filter_set['path_template']:
                        try:
                            lam, t = np.loadtxt(template.format(b)).T
                            break
                        except FileNotFoundError:
                            continue
                else:
                    lam, t = np.loadtxt(filter_set['path_template'].format(b)).T
                
                plt.plot(lam, t, label=f"{filter_set['title'].split()[0]}_{b}")
            except Exception as e:
                print(f"Could not load {b} filter: {e}")
    if filter_set["title"] == "Bessel Filters":
        plt.legend(loc=1, frameon=False, bbox_to_anchor=(0.75,1))
    elif filter_set["title"] == "SuprimeCam Filters":
        plt.legend(loc='best', frameon=False, ncols=2)
    else:
        plt.legend(loc='best', frameon=False)
    plt.ylim(bottom=0)
    
    # Set consistent x-limits
    plt.xlim(xlims)
    
    plt.xlabel(r"wavelength $[\AA]$")
    plt.ylabel("Transmission")

plt.tight_layout()
plt.savefig("filters.svg", dpi=600)